In [ ]:
# %matplotlib notebook
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

In [ ]:
def load_data(path):
    # Find the weight file in the directory
    files = [f for f in os.listdir(path) if f.startswith('weights_') and f.endswith('.txt')]
    if not files:
        raise ValueError(f"No weight files found in directory: {path}")
    # We assume there's only one file matching the pattern, hence we take the first one
    file_path = os.path.join(path, files[0])
    # Load the data using pandas
    weights = pd.read_csv(file_path, header=None, names=['weight'])
#     weights = weights[45:] # for 7,10,14,15,16 ->50,20,23,25,45
    return weights['weight']

In [ ]:
def plot(data):
    fig, axs = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    
    # Raw data plot
    axs[0].plot(data, label='Raw Data', alpha=0.5, color='green')
    axs[0].set_title('Raw Weight Data')
    axs[0].set_ylabel('Weight (grams)')
    axs[0].legend()
    plt.interactive(False)
    plt.show()

In [ ]:
def plot_ly(data):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

    # Raw data plot
    fig.add_trace(go.Scatter(y=data, mode='lines', name='Raw Data', opacity=0.5, line=dict(color='green')), row=1, col=1)

    # Update yaxis properties
    fig.update_yaxes(title_text='Weight (grams)', row=1, col=1)

    # Update titles with light mode theme color    
    fig.update_layout(height=700, width=1000, title_text='Raw Weight Data', template='plotly_white')

    fig.show()

In [ ]:
path = '../data/raw/19'
weights = load_data(path).to_numpy()

In [ ]:
plot_ly(weights)

In [ ]:
def process_meal_data(raw_data):
    processed_data = []
    last_valid_weight = raw_data[0]  # Initialize with the first measurement

    for i in range(len(raw_data)):
        if i < len(raw_data) - 1:  # Check if not at the last element
            current_weight = raw_data[i]
            next_weight = raw_data[i + 1]

            # Check for a valid decrease
            if last_valid_weight - current_weight <= 80 and \
               abs(next_weight - current_weight) <= 3:
                last_valid_weight = current_weight

        processed_data.append(last_valid_weight)

    return processed_data

In [ ]:
processed_data = process_meal_data(weights)
weights
processed_data

In [ ]:
plot_ly(processed_data)

In [ ]:
def process_meal_data_2(data, stability_range=3, max_decrease=70):
    """
    Process the meal weight data to ensure decreases are within specified limits and stable.

    Parameters:
    data (list or np.array): The raw meal weight data.
    stability_range (int): The allowed fluctuation range for considering a decrease as stable (default 3 grams).
    max_decrease (int): Maximum allowed decrease in weight between two consecutive measurements (default 40 grams).

    Returns:
    np.array: The processed meal weight data with filtered decreases.
    """
    if len(data) < 2:
        return np.array(data)  # Not enough data to process

    processed_data = np.copy(data)
    for i in range(1, len(data)):
        current_decrease = processed_data[i - 1] - processed_data[i]

        # Check if decrease is more than the maximum allowed
        if current_decrease > max_decrease:
            processed_data[i] = processed_data[i - 1]
        else:
            # Check for stability in the next measurement (if exists)
            if i < len(data) - 1:
                next_decrease = processed_data[i] - data[i + 1]
                if abs(next_decrease) > stability_range:
                    processed_data[i] = processed_data[i - 1]

    return processed_data

In [ ]:
processed_data = process_meal_data_2(weights)
weights
processed_data

In [ ]:
plot_ly(processed_data)